In [1]:
from functionUtils import *
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook, tnrange
from workalendar.america import Brazil
from datetime import date,timedelta
import pandas as pd
import numpy as np
import warnings
import workalendar
import datetime
import gc
DATA_PATH = './datasets/'
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
df_train = pd.read_csv(DATA_PATH+'train.csv')
df_test = pd.read_csv(DATA_PATH+'test.csv')
df_historical = pd.read_csv(DATA_PATH+'historical_transactions.csv',dtype={'purchase_date':np.str},low_memory=True)
df_new_merchant = pd.read_csv(DATA_PATH+'new_merchant_transactions.csv',dtype={'purchase_date':np.str},low_memory=True)

df_train['is_test'] = 0
df_test['is_test'] = 1
#用户在历史和之后的记录中是否出现过
for df in [df_train,df_test]:
    df['is_in_new_monthlag1'] = df['card_id'].isin(df_new_merchant[df_new_merchant.month_lag==1].card_id).astype(int)
    df['is_in_new_monthlag2'] = df['card_id'].isin(df_new_merchant[df_new_merchant.month_lag==2].card_id).astype(int)
df_data = pd.concat([df_train,df_test])
df_data = pd.get_dummies(df_data,columns=['feature_1','feature_2','feature_3'],prefix=['feature_1','feature_2','feature_3'])
df_transactions = pd.concat([df_historical,df_new_merchant])

In [3]:
df_transactions['purchase_amount'] = np.round(df_transactions['purchase_amount']/0.00150265118+497.06,2)

In [4]:
## id类特征
df_temp = df_historical.card_id.value_counts().reset_index()
df_temp.rename(columns={'index':'card_id','card_id':'hist_card_id_counts'},inplace=True)
df_temp['hist_card_id_ratio'] = df_temp['hist_card_id_counts']/df_historical.shape[0]
df_data = df_data.merge(df_temp,on='card_id',how='left')

df_temp = df_new_merchant.card_id.value_counts().reset_index()
df_temp.rename(columns={'index':'card_id','card_id':'new_card_id_counts'},inplace=True)
df_temp['new_card_id_ratio'] = df_temp['new_card_id_counts']/df_new_merchant.shape[0]
df_data = df_data.merge(df_temp,on='card_id',how='left')

df_data['hist/new_card_counts_ratio'] = df_data['hist_card_id_counts']/df_data['new_card_id_counts']

df_data[['hist_card_id_counts','hist_card_id_ratio','new_card_id_counts','new_card_id_ratio','hist/new_card_counts_ratio']].fillna(0,inplace=True)

del df_temp
gc.collect()

7

#### 1.1 user features:时间特征&统计特征

In [5]:
dateHandle = pd.to_datetime(df_data['first_active_month'])
df_data['active_year'] = dateHandle.dt.year
df_data['active_month'] = dateHandle.dt.month
df_data['active_to_base_time'] = (datetime.date(2018,2,28) - dateHandle.dt.date).dt.days

#hist第一次刷卡时间
df_temp = df_historical.groupby('card_id')['purchase_date'].min().reset_index()
df_temp.rename(columns={'purchase_date':'hist_first_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')
#最后一次刷卡时间
df_temp = df_historical.groupby('card_id')['purchase_date'].max().reset_index()
df_temp.rename(columns={'purchase_date':'hist_last_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')

#new第一次刷卡时间
df_temp = df_new_merchant.groupby('card_id')['purchase_date'].min().reset_index()
df_temp.rename(columns={'purchase_date':'new_first_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')
#最后一次刷卡时间
df_temp = df_new_merchant.groupby('card_id')['purchase_date'].max().reset_index()
df_temp.rename(columns={'purchase_date':'new_last_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')

#hist-new 购买时长
df_data['hist_first_to_base_time'] = (pd.to_datetime(df_data['hist_first_purchase_date']).dt.date -datetime.date(2018,2,28)).dt.days
df_data['hist_last_to_base_time'] = (pd.to_datetime(df_data['hist_last_purchase_date']).dt.date -datetime.date(2018,2,28)).dt.days
df_data['hist_last_to_first_time'] = (pd.to_datetime(df_data['hist_last_purchase_date']).dt.date - pd.to_datetime(df_data['hist_first_purchase_date']).dt.date).dt.days
df_data['hist_first_active_time'] = (pd.to_datetime(df_data['hist_first_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days
df_data['hist_last_active_time'] = (pd.to_datetime(df_data['hist_last_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days

df_data['new_first_to_base_time'] = (pd.to_datetime(df_data['new_first_purchase_date']).dt.date - datetime.date(2018,2,28)).dt.days
df_data['new_last_to_base_time'] = (pd.to_datetime(df_data['new_last_purchase_date']).dt.date - datetime.date(2018,2,28)).dt.days
df_data['new_first_to_active_time'] = (pd.to_datetime(df_data['new_first_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days
df_data['new_last_to_active_time'] = (pd.to_datetime(df_data['new_last_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days
df_data['new_last_to_first_time'] = (pd.to_datetime(df_data['new_last_purchase_date']).dt.date - pd.to_datetime(df_data['new_first_purchase_date']).dt.date).dt.days

#hist最后一次购买距离new第一次购买的时间差
df_data['new_to_hist_time'] = (pd.to_datetime(df_data['new_first_purchase_date']).dt.date - 
                               pd.to_datetime(df_data['hist_last_purchase_date']).dt.date).dt.days
#单位时间内的刷卡次数
df_data['hist_per_time_purchaseCounts'] = df_data['hist_card_id_counts']/df_data['hist_last_to_first_time']
df_data['new_per_time_purchaseCounts'] = df_data['new_card_id_counts']/df_data['new_last_to_first_time']

df_data.drop(columns=['hist_last_purchase_date','hist_first_purchase_date','new_first_purchase_date','new_last_purchase_date'],inplace=True)
df_data.replace([np.inf,-np.inf],0,inplace=True)
df_data.fillna(-1,inplace=True)
del df_train,df_test,df_historical,df_new_merchant,df_temp
gc.collect()

598

In [6]:
%%time
def dateUtils(df=None,timeCol='purchase_date'):
    dateHandle = pd.to_datetime(df[timeCol])
    df['week'] = dateHandle.dt.week
    df['year'] = dateHandle.dt.year
    df['month'] = dateHandle.dt.month
    df['dayofweek'] = dateHandle.dt.dayofweek
    df['weekend'] = (dateHandle.dt.weekday >=5).astype(int)
    df['hour'] = dateHandle.dt.hour
    df['month_gap'] = (dateHandle.dt.date - datetime.date(2018,2,28)).dt.days//30
    df['day_gap'] = (dateHandle.dt.date - datetime.date(2018,2,28)).dt.days
    #cardid用户连续购买之间的时间差
    df['purchase_time'] = dateHandle
    df['purchase_time_diff_days'] = df.groupby('card_id')['purchase_time'].apply(lambda series:series.diff(1)).dt.days
    df['purchase_time_diff_days'].fillna(df.purchase_time_diff_days.mean(),inplace=True)
    df['purchase_time_diff_seconds'] = df.groupby('card_id')['purchase_time'].apply(lambda series:series.diff(1)).dt.seconds
    df['purchase_time_diff_seconds'].fillna(df.purchase_time_diff_seconds.mean(),inplace=True)
    df.drop(columns=['purchase_time'],inplace=True)
    def getdate2Holiday(d):
        cal = Brazil()
        holiday = cal.holidays(d.year)
        dis,flag= 0,0
        year,month,day = d.year,d.month,d.day
        d = datetime.date(year,month,day)
        for i,h in enumerate(holiday):
            if holiday[i][0]>d:
                flag = 1
                dis = (holiday[i][0] - d).days
                return dis
        if flag==0:
            dis = (holiday[-1][0] - d).days
        return dis
    df['date2Holiday'] = df['purchase_date'].apply(lambda x:getdate2Holiday(pd.to_datetime(x)))
    
    return df
#缺失值处理
#C2:区域；C3分期等级；C1为-1城市编码对应Y
df_transactions['category_2'].fillna(6,inplace=True)
df_transactions['category_3'].fillna('B',inplace=True)
df_transactions['merchant_id'].fillna(df_transactions['merchant_id'].value_counts().index[0],inplace=True)
df_transactions['installments'].replace(-1,1,inplace=True)
df_transactions['installments'].replace(999,0,inplace=True)

#交互特征
df_transactions['city_C1'] = ((df_transactions['city_id']< 0) + 0).astype(np.str) + '_' + df_transactions['category_1'].astype(np.str)
df_transactions['C2_state'] = df_transactions['category_2'].astype(np.str) + '_' + df_transactions['state_id'].astype(np.str) 
df_transactions['C2_state_subsector'] = df_transactions['category_2'].astype(np.str) + '_' + df_transactions['state_id'].astype(np.str) + '_' + df_transactions['subsector_id'].astype(np.str)
df_transactions['subsector_city'] = df_transactions['subsector_id'].astype(np.str) + '_' + ((df_transactions['city_id'] < 0) + 0).astype(np.str)
df_transactions['auth_C3'] = df_transactions['authorized_flag'].astype(np.str) + '_' + df_transactions['category_3'].astype(np.str)

cateCols = ['city_C1','C2_state','C2_state_subsector','subsector_city','auth_C3']
df_transactions = label_encoding(df_transactions,cateCols)

for cate in ['A','B','C']:
    df_transactions['category_3_%s'%cate] = (df_transactions['category_3']==cate) + 0
for cate in [1,2,3,4,5,6]:
    df_transactions['category_2_%s'%cate] = (df_transactions['category_2']==cate) + 0
for cate in ['Y','N']:
    df_transactions['authorized_flag_%s'%cate] = (df_transactions['authorized_flag']==cate) + 0

df_transactions['category_1'] = df_transactions['category_1'].map({'Y':1,'N':0})
df_transactions['category_3'] = df_transactions['category_3'].map({'A':0,'B':1,'C':2})
df_transactions['authorized_flag'] = df_transactions['authorized_flag'].map({'Y':1,'N':0})

df_transactions.sort_values(by=['card_id','purchase_date'],ascending=True,inplace=True)
df_transactions = dateUtils(df_transactions,timeCol='purchase_date')
df_transactions = reduce_mem_usage(df_transactions)



Memory usage of dataframe is 9957.63 MB
Memory usage after optimization is: 2489.41 MB
Decreased by 75.0%
CPU times: user 1h 20min 22s, sys: 2min 54s, total: 1h 23min 16s
Wall time: 1h 17min 45s


In [7]:
df_transactions.sort_values(by=['card_id','purchase_date'],ascending=True,inplace=True)
# df_transactions['purchase_diff'] = df_transactions.groupby('card_id')['purchase_amount'].apply(lambda series:series.diff(1)).values
# df_transactions['purchase_diff'].fillna(0,inplace=True)
df_hist_transactions = df_transactions[df_transactions.month_lag<=0]
df_new_transactions = df_transactions[df_transactions.month_lag>0]
for df in [df_hist_transactions,df_new_transactions]:
    df['purchase_diff'] = df.groupby('card_id')['purchase_amount'].apply(lambda series:series.diff(1)).values
    df['purchase_diff'].fillna(0,inplace=True)
df_hist_auth_Y_transactions = df_hist_transactions[df_hist_transactions.authorized_flag==1]
df_hist_auth_N_transactions = df_hist_transactions[df_hist_transactions.authorized_flag==0]

In [ ]:
# def find_upperOutlier(series):
#     return (series-series.mean())>2.04*series.std()
# def find_lowerOutlier(series):
#     return (series-series.mean())<-2.04*series.std()
# def cap_values(series):
#     maxOutliers = find_upperOutlier(series)
#     minOutliers = find_lowerOutlier(series)
    
#     max_val = series[(~maxOutliers) & (~minOutliers)].max()
#     min_val = series[(~maxOutliers) & (~minOutliers)].min()
    
#     series[maxOutliers] = max_val
#     series[minOutliers] = min_val
#     return series
# #指数平均处理purchase_amount
# def calc_rollWeight_mean(series,alpha=0.98,adjust=True):
#     return series.ewm(alpha=alpha,adjust=adjust,span=7).mean()
# def targetPro(df_feature,label='purchase_amount',group='card_id'):
#     df_feature['purchase_rollweight_mean'] = df_feature.groupby(group).apply(lambda series:calc_rollWeight_mean(series[label])).values
#     return df_feature
# for df_feature in [df_hist_auth_Y_transactions,df_hist_auth_N_transactions,df_new_transactions]:
#     df_feature['purchase_is_outlier'] = df_feature.groupby('card_id').apply(lambda series:find_upperOutlier(series['purchase_amount'])).values
#     df_feature['purchase_is_outlier'] = df_feature['purchase_is_outlier'].astype(np.int)
#     df_feature['purchase_capped'] = df_feature.groupby(['card_id']).apply(lambda series:cap_values(series['purchase_amount'])).values
#     df_feature = targetPro(df_feature)


In [23]:
#用户月消费记录
def getMonthPurchase(df_data,df_feature,group='month_gap',fea='purchase_amount',name='hist'):
    df_purchase = df_feature.groupby(['card_id','month_gap'])['purchase_amount'].sum().reset_index()
    df_purchase.rename(columns={'purchase_amount':'purchase_amount_sum_month'},inplace=True)
    df_purchase.sort_values(by=['month_gap'],inplace=True,ascending=True)

    df_temp = df_purchase[['card_id','month_gap','purchase_amount_sum_month']]
    df_temp.index = df_temp.card_id
    df_temp = df_temp.set_index(['month_gap'],append=True)
    df_temp = pd.Series(df_temp['purchase_amount_sum_month'].values.reshape(len(df_temp['purchase_amount_sum_month'])),index=df_temp.index)
    df_temp = df_temp.unstack()
    df_temp.reset_index(inplace=True)
    cols = ['card_id']
    for index in list(df_purchase['month_gap'].unique()):
        cols.append('%s_month%s_purchase'%(name,index))
    df_temp.columns = cols
    df_temp.fillna(0,inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    
    del df_purchase,df_temp,cols
    gc.collect()
    
    return df_data

#周消费记录
def getWeekPurchase(df_data,df_feature,group='week',fea='purchase_amount',name='hist'):
    df_purchase = df_feature.groupby(['card_id','week'])['purchase_amount'].sum().reset_index()
    df_purchase.rename(columns={'purchase_amount':'purchase_sum_week'},inplace=True)
    df_purchase.sort_values(by=['week'],inplace=True,ascending=True)
    df_count = df_purchase.groupby(['card_id'])['purchase_sum_week'].count().reset_index()
    df_count.rename(columns={'purchase_sum_week':'%s_purchase_counts'%name},inplace=True)

    df_temp = df_purchase[['card_id','week','purchase_sum_week']]
    df_temp.index = df_temp.card_id
    df_temp = df_temp.set_index(['week'],append=True)
    df_temp = pd.Series(df_temp['purchase_sum_week'].values.reshape(len(df_temp['purchase_sum_week'])),index=df_temp.index)
    df_temp = df_temp.unstack()
    df_temp.reset_index(inplace=True)
    cols = ['card_id']
    for index in list(df_purchase['week'].unique()):
        cols.append('%s_week%s_purchase'%(name,index))
    df_temp.columns = cols
    df_temp = df_temp.merge(df_count,on='card_id',how='left')
    df_temp.fillna(0,inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    return df_data
#用户月分期记录
def getMonthInstallments(df_data,df_feature,group='month_gap',fea='installments',name='hist'):
    df_purchase = df_feature.groupby(['card_id','month_gap'])['installments'].sum().reset_index()
    df_purchase.rename(columns={'installments':'installments_sum_month'},inplace=True)
    df_purchase.sort_values(by=['month_gap'],inplace=True,ascending=True)

    df_temp = df_purchase[['card_id','month_gap','installments_sum_month']]
    df_temp.index = df_temp.card_id
    df_temp = df_temp.set_index(['month_gap'],append=True)
    df_temp = pd.Series(df_temp['installments_sum_month'].values.reshape(len(df_temp['installments_sum_month'])),index=df_temp.index)
    df_temp = df_temp.unstack()
    df_temp.reset_index(inplace=True)
    cols = ['card_id']
    for index in list(df_purchase['month_gap'].unique()):
        cols.append('%s_month%s_installments'%(name,index))
    df_temp.columns = cols
    df_temp.fillna(0,inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    del df_purchase,df_temp,cols
    gc.collect()
    return df_data
def getWeekInstallments(df_data,df_feature,group='week',fea='installments',name='hist'):
    df_purchase = df_feature.groupby(['card_id','week'])['installments'].sum().reset_index()
    df_purchase.rename(columns={'installments':'installments_sum_week'},inplace=True)
    df_purchase.sort_values(by=['week'],inplace=True,ascending=True)

    df_temp = df_purchase[['card_id','week','installments_sum_week']]
    df_temp.index = df_temp.card_id
    df_temp = df_temp.set_index(['week'],append=True)
    df_temp = pd.Series(df_temp['installments_sum_week'].values.reshape(len(df_temp['installments_sum_week'])),index=df_temp.index)
    df_temp = df_temp.unstack()
    df_temp.reset_index(inplace=True)
    cols = ['card_id']
    for index in list(df_purchase['week'].unique()):
        cols.append('%s_week%s_installments'%(name,index))
    df_temp.columns = cols
    df_temp.fillna(0,inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    del df_purchase,df_temp,cols
    gc.collect()
    return df_data

df_data = getMonthPurchase(df_data,df_hist_auth_Y_transactions,name='hist_auth_Y')
df_data = getMonthPurchase(df_data,df_hist_auth_N_transactions,name='hist_auth_N')
df_data = getMonthPurchase(df_data,df_new_transactions,name='new')
# df_data = getMonthInstallments(df_data,df_hist_transactions,name='hist')
# df_data = getMonthInstallments(df_data,df_new_transactions,name='new')


# df_data = getWeekPurchase(df_data,df_hist_auth_Y_transactions,name='hist_auth_Y')
# df_data = getWeekPurchase(df_data,df_hist_auth_N_transactions,name='hist_auth_N')
# df_data = getWeekPurchase(df_data,df_new_transactions,name='new')
# df_data = getWeekInstallments(df_data,df_hist_transactions,name='hist')
# df_data = getWeekInstallments(df_data,df_new_transactions,name='new')

df_data.replace([np.inf,-np.inf],np.nan,inplace=True)
df_data.fillna(0,inplace=True)

In [33]:
dropCols = set(df_data.columns) - set(originalCols)
df_data.drop(columns=dropCols,inplace=True)
df_data.head()

,card_id,first_active_month,is_in_new_monthlag1,is_in_new_monthlag2,is_test,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,...,new_month-7_purchase,new_month-6_purchase,new_month-5_purchase,new_month-4_purchase,new_month-3_purchase,new_month-2_purchase,new_month-1_purchase,new_month0_purchase,new_month1_purchase,new_month2_purchase
0,C_ID_92a2005557,2017-06,1,1,0,-0.820283,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1042.569946,1549.329956,26.59
1,C_ID_3d0044924f,2017-01,1,1,0,0.392913,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,23.99,29.689999,29.980000,0.00
2,C_ID_d639edf6cd,2016-08,0,1,0,0.688056,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,31.000000,0.00
3,C_ID_186d6a6901,2017-09,1,1,0,0.142495,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,140.880005,241.100006,0.00
4,C_ID_cdbd2c0db2,2017-11,1,1,0,-0.159749,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1305.390015,3328.050049,0.00


In [ ]:
%%time
originalCols = list(df_data.columns)
###持卡人不同授权类型的购买总量
def getAuthorizedPurchase(df_data,df_features,name='hist'):
    df_temp = df_features.groupby(['card_id','authorized_flag'])['purchase_amount'].sum().reset_index()
    df_temp = df_temp.pivot(index='card_id',columns='authorized_flag',values='purchase_amount')
    cols = []
    for col in df_temp.columns:
        cols.append(name + df_temp.columns.name+'_'+ np.str(col)+'_purchasesum')
    df_temp.columns = cols
    df_temp.columns.name=None
    df_temp.reset_index(inplace=True)
    df_temp.fillna(0,inplace=True)
    if name=='hist':
        df_temp['%s_Auth_purchase_sum'%name] = df_temp[cols[0]] + df_temp[cols[1]]
        df_temp[cols[0]+'_ratio'] = df_temp[cols[0]]/df_temp['%s_Auth_purchase_sum'%name]
        df_temp[cols[1]+'_ratio'] = df_temp[cols[1]]/df_temp['%s_Auth_purchase_sum'%name]
        df_temp.drop(columns=['%s_Auth_purchase_sum'%name],inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    del df_temp
    gc.collect()
    return df_data
#### 用户对商家的回购率
def getUserRepurchaseRatio(df_data,df_fea,group=['card_id','merchant_id'],fea='purchase_amount',name='repurchase_mechant_ratio'):
    df_temp = df_features.groupby(group)[fea].count().reset_index().rename(
        columns={fea:'purchase_amount_counts'})
    df_temp['repurchase_flag'] = (df_temp['purchase_amount_counts']>1) + 0
    df = df_temp.groupby(['card_id'])['purchase_amount_counts'].sum().reset_index().rename(columns={'purchase_amount_counts':'purchase_records'})
    df_temp['repurchase_counts'] = df_temp['repurchase_flag'] * df_temp['purchase_amount_counts']
    df_temp = df_temp.merge(df,on='card_id',how='left')
    df_temp['repurchase_ratio'] = df_temp['repurchase_counts']/df_temp['purchase_records']
    df_temp = df_temp.groupby('card_id')['repurchase_ratio'].max().reset_index().rename(columns={'repurchase_ratio':name})
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    del df,df_temp
    gc.collect()
    return df_data

for flag,df_features in zip(['hist_auth_Y','hist_auth_N','new'],[df_hist_auth_Y_transactions,df_hist_auth_N_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    if 'hist' in flag:
        month_lag = [-1,-3,-5,-7,-13]
    else:
        month_lag = [1,2]
    for index in month_lag:
        df = df_features[df_features.month_lag>=index]
        #各个阶段的purchase_amount(强特)
        df_data = getMeanStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountMean_%s'%(flag,index))
        df_data = getMaxStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountMax_%s'%(flag,index))
        df_data = getMedianStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountMedian_%s'%(flag,index))
        df_data = getSumStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountSum_%s'%(flag,index))
        df_data = getStdStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purcahseAmountStd_%s'%(flag,index))
        df_data = getCountsStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purcahseAmountCount_%s'%(flag,index))
        
        df_data['%s_per_time_purchaseAmountSum_%s'%(flag,index)] = df_data['%s_purchaseAmountSum_%s'%(flag,index)]/(np.abs(index))      
        df_data['%s_per_time_purcahseAmountCount_%s'%(flag,index)] = df_data['%s_purcahseAmountCount_%s'%(flag,index)]/(np.abs(index))

        #分期数
        df_data = getMeanStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsMean_%s'%(flag,index))
        df_data = getMaxStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsMax_%s'%(flag,index))
        df_data = getSumStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsSum_%s'%(flag,index))
        df_data = getCountsStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsCount_%s'%(flag,index))
        
        df_data['%s_per_time_installmentsSum_%s'%(flag,index)] = df_data['%s_installmentsSum_%s'%(flag,index)]/(np.abs(index))
        df_data['%s_per_time_installmentsCount_%s'%(flag,index)] = df_data['%s_installmentsCount_%s'%(flag,index)]/(np.abs(index))

        #purchase_diff(强特)
        df_data = getMeanStaticsFeatures(df_data,df,['card_id'],'purchase_diff',name='%s_purchaseDiffMean_%s'%(flag,index))
        df_data = getMaxStaticsFeatures(df_data,df,['card_id'],'purchase_diff',name='%s_purchaseDiffMax_%s'%(flag,index))
        df_data = getMedianStaticsFeatures(df_data,df,['card_id'],'purchase_diff',name='%s_purchaseDiffMedian_%s'%(flag,index))
        df_data = getMinStaticsFeatures(df_data,df,['card_id'],'purchase_diff',name='%s_purchaseDiffMin_%s'%(flag,index))
        df_data = getStdStaticsFeatures(df_data,df,['card_id'],'purchase_diff',name='%s_purcahseDiffStd_%s'%(flag,index))
        
        #day_diff
#         df_data = getMaxStaticsFeatures(df_data,df_features,['card_id'],'day_diff',name='%s_dayDiffMax_%s'%(flag,index))
#         df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'day_diff',name='%s_dayDiffMin_%s'%(flag,index))
#         df_data = getCategoryFrequenceMax(df_data,df_features,['card_id'],'day_diff',name='%s_dayDiffFrequenceMax_%s'%(flag,index))
          
        #信用卡分期类别特征
        for cate in ['category_3_A','category_3_B','category_3_C']:
            df_data = getSumStaticsFeatures(df_data,df,['card_id'],cate,name='%s_%s_sum_%s'%(flag,cate,index))

    if 'hist' in flag:
        histflag = 'hist'
    else:
        histflag = 'new'
    #信用卡授权统计
    for cate in ['Y','N']:
        df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],'authorized_flag_%s'%cate,
                                        name='%s_authorized_flag_%s_sum_%s'%(flag,cate,index))
        df_data['%s_per_time_authorized_flag_%s_sum_%s'%(flag,cate,index)] = (
            df_data['%s_authorized_flag_%s_sum_%s'%(flag,cate,index)]/df_data['%s_last_to_first_time'%histflag])
    
    ##date2Holiday
    df_data = getMaxStaticsFeatures(df_data,df_features,['card_id'],'date2Holiday',name='%s_date2HolidayMax'%flag)
    df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'date2Holiday',name='%s_date2HolidayMin'%flag)
    df_data = getMeanStaticsFeatures(df_data,df_features,['card_id'],'date2Holiday',name='%s_date2Holiday'%flag)

    #AuthorizedPurchase
    df_data = getAuthorizedPurchase(df_data,df_features,name='%s_'%flag)
    #purchase_is_outlier
#     df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],'purchase_is_outlier',name='%s_purchaseOutlier_sum'%flag)
    #month
    df_data = getCategoryCounts(df_data,df_features,['card_id'],'month',name='%s_activeMonthCount'%flag)
    df_data = getCategoryFrequenceMax(df_data,df_features,['card_id'],'month',name='%s_activeMonthMax'%flag)
    df_data = getCategoryFrequenceMaxRatio(df_data,df_features,['card_id'],'month',name='%s_activeMonthMaxRatio'%flag)
    df_data['%s_per_time_activeMonthCount'%flag] = (df_data['%s_activeMonthCount'%flag]/
                                                    df_data['%s_last_to_first_time'%histflag])
    #month_lag
    df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'month_lag',name='%s_monthLagMin'%flag)
    df_data = getMaxStaticsFeatures(df_data,df_features,['card_id'],'month_lag',name='%s_monthLagMax'%flag)
    #month_gap
    df_data = getCategoryCounts(df_data,df_features,['card_id'],'month_gap',
                                name='%s_monthGap_categoryCounts'%flag)
    df_data = getCategoryCountsRatio(df_data,df_features,['card_id'],'month_gap',
                                     name='%s_monthGap_categoryCountsRatio'%flag)

    #day_gap
    df_data = getMaxStaticsFeatures(df_data,df_features,['card_id'],'day_gap',name='%s_dayGapMax'%flag)
    df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'day_gap',name='%s_dayGapMin'%flag)
    #回购率
    df_data = getUserRepurchaseRatio(df_data,df_features,name='%s_repurchase_merchant_ratio'%flag)
    df_data = getUserRepurchaseRatio(df_data,df_features,group=['card_id','merchant_category_id'],name='%s_repurchase_merCate_ratio'%flag)
    #purchase shift比例
    df_features['%s_purchase_shift'%flag] = df_features.groupby(['card_id'])['purchase_amount'].apply(lambda series:series.shift(1)).values
    df_features['%s_purchase_shift'%flag].fillna(0,inplace=True)
    df_features['%s_purchase_shift_ratio'%flag] = df_features['%s_purchase_shift'%flag]/df_features['purchase_amount']
    df_temp = df_features.groupby(['card_id'])['%s_purchase_shift_ratio'%flag].max().reset_index()
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    #hour
    df_data = getMeanStaticsFeatures(df_data,df_features,['card_id'],'hour',name='%s_HourMean_'%(flag))
    df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],'hour',name='%s_HourSum_'%(flag))
    df_data = getCountsStaticsFeatures(df_data,df_features,['card_id'],'hour',name='%s_HourCounts_'%(flag))
    #weekend
    df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],'weekend',name='%s_weekendSum_'%(flag))
    #dayofweek
    df_data = getMeanStaticsFeatures(df_data,df_features,['card_id'],'dayofweek',name='%s_dayofweekMean_'%flag)
    df_data = getCategoryCounts(df_data,df_features,['card_id'],'dayofweek',name='%s_dayofWeekCounts_'%flag)
    
    for cate in [1,2,3,4,5,6]:
        df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],'category_2_%s'%cate,
                                        name='%s_category_2_%s_sum'%(flag,cate))
        df_data['%s_per_time_category_2_%s_sum'%(flag,cate)] = (
            df_data['%s_category_2_%s_sum'%(flag,cate)]/df_data['%s_last_to_first_time'%histflag])
        
    categoryCols = cateCols + ['authorized_flag','merchant_id','city_id','category_1','category_2','category_3',
                               'merchant_category_id','subsector_id']
    #purchase_time_diff
    df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'purchase_time_diff_days',name='%s_purchase_time_diff_days_min_'%(flag))
    df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'purchase_time_diff_seconds',name='%s_purchase_time_diff_seconds_min_'%(flag))
    
    for fea in ['purchase_time_diff_seconds','purchase_time_diff_days']:
        df_data = getCategoryFrequenceMax(df_data,df_features,['card_id'],fea,
                                          name='%s_%s_frequenceMax'%(flag,fea))
        df_data = getCategoryCounts(df_data,df_features,['card_id'],fea,
                                    name='%s_%s_categoryCounts'%(flag,fea))
        df_data = getCategoryFrequenceMaxRatio(df_data,df_features,['card_id'],fea,
                                               name='%s_%s_frequenceMaxRatio'%(flag,fea))
        df_data = getCategoryCountsRatio(df_data,df_features,['card_id'],fea,
                                         name='%s_%s_categoryCountsRatio'%(flag,fea))
    for fea in categoryCols:
        df_data = getCategoryFrequenceMax(df_data,df_features,['card_id'],fea,
                                          name='%s_%s_frequenceMax'%(flag,fea))
        df_data = getCategoryCounts(df_data,df_features,['card_id'],fea,
                                    name='%s_%s_categoryCounts'%(flag,fea))
        df_data = getCategoryFrequenceMaxRatio(df_data,df_features,['card_id'],fea,
                                               name='%s_%s_frequenceMaxRatio'%(flag,fea))
        df_data = getCategoryCountsRatio(df_data,df_features,['card_id'],fea,
                                         name='%s_%s_categoryCountsRatio'%(flag,fea))

#month_lag=0的消费平均消费水平
df_temp = df_hist_transactions[df_hist_transactions.month_lag==0]
df_temp = df_temp.groupby(['card_id'])['purchase_amount'].mean().reset_index().rename(
    columns={'purchase_amount':'hist_purchaseAmountMean_0'}) 
df_data = df_data.merge(df_temp,on='card_id',how='left')

del df_temp,df_features,df_hist_transactions,df_new_transactions
gc.collect()

In [35]:
df_data['avg_purchase_ratio_-1'] = df_data['hist_auth_Y_purchaseAmountMean_-1']/df_data['hist_purchaseAmountMean_0']
df_data['avg_purchase_ratio_-3'] = df_data['hist_auth_Y_purchaseAmountMean_-3']/df_data['hist_purchaseAmountMean_0']
df_data['avg_purchase_ratio_-5'] = df_data['hist_auth_Y_purchaseAmountMean_-5']/df_data['hist_purchaseAmountMean_0']
df_data['avg_purchase_ratio_-7'] = df_data['hist_auth_Y_purchaseAmountMean_-7']/df_data['hist_purchaseAmountMean_0']
df_data['avg_purchase_ratio_-13'] = df_data['hist_auth_Y_purchaseAmountMean_-13']/df_data['hist_purchaseAmountMean_0']
df_data['avg_purchase_ratio_1'] = df_data['new_purchaseAmountMean_1']/df_data['hist_purchaseAmountMean_0']
df_data['avg_purchase_ratio_2'] = df_data['new_purchaseAmountMean_2']/df_data['hist_purchaseAmountMean_0']

df_data.replace([np.inf,-np.inf],np.nan,inplace=True)
df_data.fillna(0,inplace=True)

In [37]:
#保存用户统计特征
df_data = reduce_mem_usage(df_data)
df_transactions = reduce_mem_usage(df_transactions)

Memory usage of dataframe is 1391.48 MB
Memory usage after optimization is: 505.74 MB
Decreased by 63.7%
Memory usage of dataframe is 2489.41 MB
Memory usage after optimization is: 2489.41 MB
Decreased by 0.0%


In [45]:
df_data.to_csv('./datasets/df_data.csv',index=False)
df_transactions.to_csv('./datasets/df_transactions.csv',index=False)

----------------------------------

In [10]:
df_uid = pd.read_csv('./datasets/df_data.csv',usecols=['card_id'])
df_transactions = pd.read_csv('./datasets/df_transactions.csv')

In [11]:
cateCols = ['city_C1','C2_state','C2_state_subsector','subsector_city','auth_C3']
categoryCols = cateCols + ['authorized_flag','merchant_id','city_id','category_1','category_2','category_3',
                           'merchant_category_id','subsector_id']
df_transactions = label_encoding(df_transactions,['merchant_id'])
df_hist_transactions = df_transactions[df_transactions.month_lag<=0]
df_new_transactions = df_transactions[df_transactions.month_lag>0]

for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    #categoryCols
    for fea in categoryCols:
        df_uid = getMeanStaticsFeatures(df_uid,df_features,['card_id'],fea,name='%s_%s_Mean'%(flag,fea))
        df_uid = getMaxStaticsFeatures(df_uid,df_features,['card_id'],fea,name='%s_%s_Max'%(flag,fea))
        df_uid = getMedianStaticsFeatures(df_uid,df_features,['card_id'],fea,name='%s_%s_Median'%(flag,fea))
        df_uid = getSumStaticsFeatures(df_uid,df_features,['card_id'],fea,name='%s_%s_Sum'%(flag,fea))
        df_uid = getStdStaticsFeatures(df_uid,df_features,['card_id'],fea,name='%s_%s_Std'%(flag,fea))
        df_uid = getCountsStaticsFeatures(df_uid,df_features,['card_id'],fea,name='%s_%s_Count'%(flag,fea))
        
df_uid.to_csv('./datasets/df_cate_statics.csv',index=False)


.................hist......................
.................new......................


In [6]:
df_hist = df_hist_transactions.groupby(['card_id','date2Holiday'])['purchase_amount'].sum().reset_index()
df_hist = df_hist.groupby('card_id')['purchase_amount'].max().reset_index().rename(columns={'purchase_amount':'hist_holiday_purchase_Max'})

df_new = df_new_transactions.groupby(['card_id','date2Holiday'])['purchase_amount'].sum().reset_index()
df_new = df_new.groupby('card_id')['purchase_amount'].max().reset_index().rename(columns={'purchase_amount':'new_holiday_purchase_Max'})

df_temp = pd.merge(df_hist,df_new,on='card_id',how='left')
df_temp['new/hist_holiday_purchase_Max_ratio'] = df_temp['new_holiday_purchase_Max']/df_temp['hist_holiday_purchase_Max']
df_temp.to_csv('./datasets/feature2/df_new_hist_holiday_purchase_Max_ratio.csv',index=False)

In [ ]:
#new/hist 
for col in ['merchant_id','merchant_category_id','city_id','subsector_id']:
    print(col)
    df_hist_temp = df_hist_transactions.groupby('card_id')[col].unique().apply(lambda x:len(x)).reset_index().rename(columns={col:'hist_%s_counts'%col})
    df_new_temp = df_new_transactions.groupby('card_id')[col].unique().apply(lambda x:len(x)).reset_index().rename(columns={col:'new_%s_counts'%col})
    df_temp = pd.merge(df_hist_temp,df_new_temp,on='card_id',how='left')
    df_temp['new/hist_%s_counts_ratio'%col] = df_temp['new_%s_counts'%col]/df_temp['hist_%s_counts'%col]
    df_temp.drop(columns=['hist_%s_counts'%col,'new_%s_counts'%col],inplace=True)
    df_temp.fillna(0,inplace=True)
    df_temp.to_csv('./datasets/feature2/df_new_hist_%s_counts_ratio.csv'%col,index=False)

merchant_id


In [ ]:
for col in ['installments','authorized_flag']:
    df_hist_temp = df_hist_transactions.groupby('card_id')[col].sum().reset_index().rename(columns={col:'hist_%s_sum'%col})
    df_new_temp = df_new_transactions.groupby('card_id')[col].sum().reset_index().rename(columns={col:'new_%s_sum'%col})
    df_temp = pd.merge(df_hist_temp,df_new_temp,on='card_id',how='left')
    df_temp['new/hist_%s_sum_ratio'%col] = df_temp['new_%s_sum'%col]/df_temp['hist_%s_sum'%col]
    df_temp.drop(columns=['hist_%s_sum'%col,'new_%s_sum'%col],inplace=True)
    df_temp.fillna(0,inplace=True)
    df_temp.to_csv('./datasets/feature2/df_new_hist_%s_sum_ratio.csv'%col,index=False)
    
del df_hist_temp,df_new_temp,df_temp

In [ ]:
df_hist = df_hist_transactions.groupby(['card_id','merchant_category_id'])['purchase_amount'].mean().reset_index()
df_hist = df_hist.groupby('card_id')['purchase_amount'].max().reset_index().rename(columns={'purchase_amount':'hist_card_and_merchant_purchase_mean'})
df_new = df_new_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_new = df_new.groupby('card_id')['purchase_amount'].max().reset_index().rename(columns={'purchase_amount':'new_card_and_merchant_purchase_mean'})
df_temp = pd.merge(df_hist,df_new,on='card_id',how='left')
df_temp['new/hist_card_and_merchant_category_id_purchase_mean_raio'] = df_temp['new_card_and_merchant_purchase_mean']/df_temp['hist_card_and_merchant_purchase_mean']
df_temp.drop(columns=['hist_card_and_merchant_purchase_mean','new_card_and_merchant_purchase_mean'],inplace=True)
df_temp.fillna(0,inplace=True)
df_temp.to_csv('./datasets/feature2/df_new_and_merchant_category_id_purchase_mean_ratio.csv',index=False)

df_hist = df_hist_transactions.groupby(['card_id','merchant_category_id'])['purchase_amount'].mean().reset_index()
df_hist = df_hist.groupby('card_id')['purchase_amount'].min().reset_index().rename(columns={'purchase_amount':'hist_card_and_merchant_purchase_mean'})
df_new = df_new_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_new = df_new.groupby('card_id')['purchase_amount'].min().reset_index().rename(columns={'purchase_amount':'new_card_and_merchant_purchase_mean'})
df_temp = pd.merge(df_hist,df_new,on='card_id',how='left')
df_temp['new/hist_card_and_merchant_category_id_purchase_min_raio'] = df_temp['new_card_and_merchant_purchase_mean']/df_temp['hist_card_and_merchant_purchase_mean']
df_temp.drop(columns=['hist_card_and_merchant_purchase_mean','new_card_and_merchant_purchase_mean'],inplace=True)
df_temp.fillna(0,inplace=True)
df_temp.to_csv('./datasets/feature2/df_new_and_merchant_category_id_purchase_min_ratio.csv',index=False)

df_hist = df_hist_transactions.groupby(['card_id','merchant_category_id'])['purchase_amount'].mean().reset_index()
df_hist = df_hist.groupby('card_id')['purchase_amount'].median().reset_index().rename(columns={'purchase_amount':'hist_card_and_merchant_purchase_mean'})
df_new = df_new_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_new = df_new.groupby('card_id')['purchase_amount'].median().reset_index().rename(columns={'purchase_amount':'new_card_and_merchant_purchase_mean'})
df_temp = pd.merge(df_hist,df_new,on='card_id',how='left')
df_temp['new/hist_card_and_merchant_category_id_purchase_median_raio'] = df_temp['new_card_and_merchant_purchase_mean']/df_temp['hist_card_and_merchant_purchase_mean']
df_temp.drop(columns=['hist_card_and_merchant_purchase_mean','new_card_and_merchant_purchase_mean'],inplace=True)
df_temp.fillna(0,inplace=True)
df_temp.to_csv('./datasets/feature2/df_new_and_merchant_category_id_purchase_median_ratio.csv',index=False)


In [ ]:
#new/hist (card,merchant) ratio
df_hist = df_hist_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_hist = df_hist.groupby('card_id')['purchase_amount'].max().reset_index().rename(columns={'purchase_amount':'hist_card_and_merchant_purchase_mean'})
df_new = df_new_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_new = df_new.groupby('card_id')['purchase_amount'].max().reset_index().rename(columns={'purchase_amount':'new_card_and_merchant_purchase_mean'})
df_temp = pd.merge(df_hist,df_new,on='card_id',how='left')
df_temp['new/hist_card_and_merchant_purchase_mean_raio'] = df_temp['new_card_and_merchant_purchase_mean']/df_temp['hist_card_and_merchant_purchase_mean']
df_temp.drop(columns=['hist_card_and_merchant_purchase_mean','new_card_and_merchant_purchase_mean'],inplace=True)
df_temp.fillna(0,inplace=True)
df_temp.to_csv('./datasets/feature2/df_new_and_merchant_purchase_mean_ratio.csv',index=False)


df_hist = df_hist_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_hist = df_hist.groupby('card_id')['purchase_amount'].min().reset_index().rename(columns={'purchase_amount':'hist_card_and_merchant_purchase_mean'})
df_new = df_new_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_new = df_new.groupby('card_id')['purchase_amount'].min().reset_index().rename(columns={'purchase_amount':'new_card_and_merchant_purchase_mean'})
df_temp = pd.merge(df_hist,df_new,on='card_id',how='left')
df_temp['new/hist_card_and_merchant_purchase_min_raio'] = df_temp['new_card_and_merchant_purchase_mean']/df_temp['hist_card_and_merchant_purchase_mean']
df_temp.drop(columns=['hist_card_and_merchant_purchase_mean','new_card_and_merchant_purchase_mean'],inplace=True)
df_temp.fillna(0,inplace=True)
df_temp.to_csv('./datasets/feature2/df_new_and_merchant_purchase_min_ratio.csv',index=False)


df_hist = df_hist_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_hist = df_hist.groupby('card_id')['purchase_amount'].median().reset_index().rename(columns={'purchase_amount':'hist_card_and_merchant_purchase_mean'})
df_new = df_new_transactions.groupby(['card_id','merchant_id'])['purchase_amount'].mean().reset_index()
df_new = df_new.groupby('card_id')['purchase_amount'].median().reset_index().rename(columns={'purchase_amount':'new_card_and_merchant_purchase_mean'})
df_temp = pd.merge(df_hist,df_new,on='card_id',how='left')
df_temp['new/hist_card_and_merchant_purchase_median_raio'] = df_temp['new_card_and_merchant_purchase_mean']/df_temp['hist_card_and_merchant_purchase_mean']
df_temp.drop(columns=['hist_card_and_merchant_purchase_mean','new_card_and_merchant_purchase_mean'],inplace=True)
df_temp.fillna(0,inplace=True)
df_temp.to_csv('./datasets/feature2/df_new_and_merchant_purchase_median_ratio.csv',index=False)

In [ ]:
#new/hist ratio
month_lag = [0,-1,-2,-3,-4,-13]
for lag in month_lag:
    df_feature = df_hist_transactions[df_hist_transactions.month_lag>=lag]
    df_hist_temp = df_feature.groupby('card_id')['purchase_amount'].sum().reset_index().rename(columns={'purchase_amount':'hist_purchase_amount_sum'})
    df_new_temp = df_new_transactions.groupby('card_id')['purchase_amount'].sum().reset_index().rename(columns={'purchase_amount':'new_purchase_amount_sum'})
    df_temp = pd.merge(df_hist_temp,df_new_temp,on='card_id',how='left')
    df_temp['new/hist_purchase_amount_sum_ratio_%s'%lag] = df_temp['new_purchase_amount_sum']/df_temp['hist_purchase_amount_sum']
    df_temp.drop(columns=['hist_purchase_amount_sum','new_purchase_amount_sum'],inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    df_data['new/hist_purchase_amount_sum_ratio_%s'%lag].fillna(0,inplace=True)

In [ ]:
#不同分期付款购买的金额
for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    df_features['install_flag'] = (df_features['installments']>0).astype(np.int)
    df_temp = df_features.groupby(['card_id','install_flag'])['purchase_amount'].sum().reset_index()
    df_temp = df_temp.pivot(index='card_id',columns='install_flag',values='purchase_amount')
    df_temp.columns.name=None
    df_temp.columns = ['%s_installments_0'%flag,'%s_installments_1'%flag]
    df_temp.reset_index(inplace=True)
    df_temp.fillna(0,inplace=True)
    df_temp['purchase_amount'] = df_temp['%s_installments_0'%flag] + df_temp['%s_installments_1'%flag]
    df_temp['%s_purchase_install_0_ratio'%flag] = df_temp['%s_installments_0'%flag]/df_temp['purchase_amount']
    df_temp['%s_purchase_install_1_ratio'%flag] = df_temp['%s_installments_1'%flag]/df_temp['purchase_amount']
    df_temp.drop(columns=['purchase_amount'],inplace=True)

    df_data = df_data.merge(df_temp,on='card_id',how='left')


In [ ]:
#不同分期等级购买的金额
for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    df_temp = df_features.groupby(['card_id','category_3'])['purchase_amount'].sum().reset_index()
    df_temp = df_temp.pivot(index='card_id',columns='category_3',values='purchase_amount')
    df_temp.columns.name=None
    cols = []
    for col in df_temp.columns:
        cols.append('%s_category_3_'%flag + np.str(col))
    df_temp.columns = cols
    df_temp.fillna(0,inplace=True)
    df_temp['purchase_amount'] = 0
    for col in cols:
        df_temp['purchase_amount'] += df_temp[col]
    for col in cols:
        df_temp[col+'_ratio'] = df_temp[col]/df_temp['purchase_amount']
    df_temp.reset_index(inplace=True)
    df_temp.drop(columns=['purchase_amount'],inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')

In [ ]:
#不同category_1购买的金额
for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    df_temp = df_features.groupby(['card_id','category_1'])['purchase_amount'].sum().reset_index()
    df_temp = df_temp.pivot(index='card_id',columns='category_1',values='purchase_amount')
    df_temp.columns.name=None
    cols = []
    for col in df_temp.columns:
        cols.append('%s_category_1_'%flag + np.str(col))
    df_temp.columns = cols
    df_temp['purchase_amount'] = 0
    df_temp.fillna(0,inplace=True)
    for col in cols:
        df_temp['purchase_amount']+=df_temp[col]
    for col in cols:
        df_temp[col+'_ratio'] = df_temp[col]/df_temp['purchase_amount']
        
    df_temp.reset_index(inplace=True)
    df_temp.drop(columns=['purchase_amount'],inplace=True)

    df_data = df_data.merge(df_temp,on='card_id',how='left')

In [ ]:
#不同category_2购买的金额
for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    df_temp = df_features.groupby(['card_id','category_2'])['purchase_amount'].sum().reset_index()
    df_temp = df_temp.pivot(index='card_id',columns='category_2',values='purchase_amount')
    df_temp.columns.name=None
    cols = []
    for col in df_temp.columns:
        cols.append('%s_category_2_'%flag + np.str(col))
    df_temp.columns = cols
    df_temp['purchase_amount'] = 0
    df_temp.fillna(0,inplace=True)
    for col in cols:
        df_temp['purchase_amount']+=df_temp[col]
    for col in cols:
        df_temp[col+'_ratio'] = df_temp[col]/df_temp['purchase_amount']
        
    df_temp.reset_index(inplace=True)
    df_temp.drop(columns=['purchase_amount'],inplace=True)

    df_data = df_data.merge(df_temp,on='card_id',how='left')

In [ ]:
#不同subsector_id购买的金额
for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    df_temp = df_features.groupby(['card_id','subsector_id'])['purchase_amount'].sum().reset_index()
    df_temp = df_temp.pivot(index='card_id',columns='subsector_id',values='purchase_amount')
    df_temp.columns.name=None
    cols = []
    for col in df_temp.columns:
        cols.append('%s_subsector_id_'%flag + np.str(col))
    df_temp.columns = cols
    df_temp['purchase_amount'] = 0
    df_temp.fillna(0,inplace=True)
    for col in cols:
        df_temp['purchase_amount']+=df_temp[col]
    for col in cols:
        df_temp[col+'_ratio'] = df_temp[col]/df_temp['purchase_amount']
        
    df_temp.reset_index(inplace=True)
    df_temp.drop(columns=['purchase_amount'],inplace=True)

    df_data = df_data.merge(df_temp,on='card_id',how='left')

In [ ]:
df_merchant = pd.read_csv('./datasets/merchants.csv',usecols=['category_4','merchant_id'])
df_transactions = df_transactions.merge(df_merchant,on='merchant_id',how='left')
df_hist_transactions = df_transactions[df_transactions.month_lag<=0]
df_new_transactions = df_transactions[df_transactions.month_lag>0]

In [ ]:
#不同category_4购买的金额
for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    df_temp = df_features.groupby(['card_id','category_4'])['purchase_amount'].sum().reset_index()
    df_temp = df_temp.pivot(index='card_id',columns='category_4',values='purchase_amount')
    df_temp.columns.name=None
    cols = []
    for col in df_temp.columns:
        cols.append('%s_category_4_'%flag + np.str(col))
    df_temp.columns = cols
    df_temp['purchase_amount'] = 0
    df_temp.fillna(0,inplace=True)
    for col in cols:
        df_temp['purchase_amount']+=df_temp[col]
    for col in cols:
        df_temp[col+'_ratio'] = df_temp[col]/df_temp['purchase_amount']
        
    df_temp.reset_index(inplace=True)
    df_temp.drop(columns=['purchase_amount'],inplace=True)

    df_data = df_data.merge(df_temp,on='card_id',how='left')

In [ ]:
df_merchant = pd.read_csv('./datasets/merchants.csv',usecols=['merchant_group_id','merchant_id'])
df_transactions = df_transactions.merge(df_merchant,on='merchant_id',how='left')
df_hist_transactions = df_transactions[df_transactions.month_lag<=0]
df_new_transactions = df_transactions[df_transactions.month_lag>0]

In [51]:
#获取词向量
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser
def getWord2Vec(df_temp=None,fea=None,embedding_size=10,name=None):  
    corpus = df_temp[fea].values
    model = Word2Vec(corpus, size=embedding_size, window=3, min_count=1, workers=16)
    Q_VEC = np.zeros((len(corpus),embedding_size))
    cols = []
    for i in range(embedding_size):
        cols.append(name+'_vec_%s'%i)
    for i in range(df_temp.shape[0]):
        Q_VEC[i,:] = np.mean(model.wv[corpus[i]],axis=0)
    df_vec = pd.DataFrame(data=Q_VEC,columns=cols)
    df_vec['card_id'] = df_temp['card_id'].values
    return df_vec
#购买次数序列构成的embedding
df_temp = df_transactions.groupby(['card_id','month'])['purchase_amount'].count().reset_index()
df_temp['purchase_amount'] = df_temp['purchase_amount'].astype(np.str)
df_temp = df_temp.groupby(['card_id'])['purchase_amount'].apply(lambda series:list(series)).reset_index()
df_temp.rename(columns={'purchase_amount':'purchase_sequence'},inplace=True)
df_vec = getWord2Vec(df_temp,fea='purchase_sequence',name='purchase_sequence')

df_data = df_data.merge(df_vec,on='card_id',how='left')


In [52]:
for flag,df_features in zip(['hist','new'],[df_hist_transactions,df_new_transactions]):
    print(".................%s......................"%flag)
    df_temp = df_features.groupby(['card_id','month'])['merchant_group_id'].count().reset_index()
    df_temp['merchant_group_id'] = df_temp['merchant_group_id'].astype(np.str)
    df_temp = df_temp.groupby(['card_id'])['merchant_group_id'].apply(lambda series:list(series)).reset_index()
    df_temp.rename(columns={'merchant_group_id':'merchant_group_id_sequence'},inplace=True)
    df_vec = getWord2Vec(df_temp,fea='merchant_group_id_sequence',name='%s_merchant_group_id_sequence'%flag)
    df_data = df_data.merge(df_vec,on='card_id',how='left')


.................hist......................
.................new......................


In [53]:
df_data = reduce_mem_usage(df_data)

Memory usage of dataframe is 864.63 MB
Memory usage after optimization is: 710.64 MB
Decreased by 17.8%


In [54]:
df_data.head()

,card_id,first_active_month,is_in_new_monthlag1,is_in_new_monthlag2,is_test,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,...,new_merchant_group_id_sequence_vec_0,new_merchant_group_id_sequence_vec_1,new_merchant_group_id_sequence_vec_2,new_merchant_group_id_sequence_vec_3,new_merchant_group_id_sequence_vec_4,new_merchant_group_id_sequence_vec_5,new_merchant_group_id_sequence_vec_6,new_merchant_group_id_sequence_vec_7,new_merchant_group_id_sequence_vec_8,new_merchant_group_id_sequence_vec_9
0,C_ID_92a2005557,2017-06,1,1,0,-0.820312,0,0,0,0,...,0.345947,-0.062195,0.024551,-0.624512,0.142944,0.300781,0.077881,-0.437988,-0.202759,-0.130493
1,C_ID_3d0044924f,2017-01,1,1,0,0.392822,0,0,0,1,...,0.911133,0.741211,0.072083,-0.093323,-0.099060,-0.074829,-0.267090,-0.425781,-0.154419,0.180054
2,C_ID_d639edf6cd,2016-08,0,1,0,0.687988,0,1,0,0,...,1.324219,1.047852,0.030182,0.047150,-0.334473,-0.287598,-0.154297,-0.343750,-0.038757,0.212402
3,C_ID_186d6a6901,2017-09,1,1,0,0.142456,0,0,0,1,...,0.891113,0.693359,0.063538,-0.142822,-0.139404,-0.051788,-0.203369,-0.373291,-0.091736,0.136108
4,C_ID_cdbd2c0db2,2017-11,1,1,0,-0.159790,1,0,0,0,...,0.345947,-0.191528,-0.007568,-0.826660,0.118042,0.407715,0.225952,-0.532715,-0.259521,-0.088623


In [55]:
df_data.to_csv('./datasets/df_data.csv',index=False)

In [2]:
##additional features
cols = ['card_id','first_active_month','feature_1','feature_2','feature_3']
df_train = pd.read_csv('./datasets/train.csv',usecols=cols+['target'])
df_test = pd.read_csv('./datasets/test.csv',usecols=cols)
df = pd.concat([df_train,df_test])
df['first_active_month'] = pd.to_datetime(df['first_active_month'])
df['quarter'] = df['first_active_month'].dt.quarter
df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days

df['days_feature1'] = df['elapsed_time'] * df['feature_1']
df['days_feature2'] = df['elapsed_time'] * df['feature_2']
df['days_feature3'] = df['elapsed_time'] * df['feature_3']

df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']

df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
df['feature_mean'] = df['feature_sum']/3
df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

df.drop(columns=['first_active_month','target'],inplace=True)

In [3]:
df.to_csv('./datasets/df_train_test_features_additional.csv',index=False)

In [10]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
def historical_transactions(num_rows=None):
    print('......historical......')
    # load csv
    hist_df = pd.read_csv('./datasets/historical_transactions.csv', nrows=num_rows)

    # fillna
    hist_df['category_2'].fillna(1.0,inplace=True)
    hist_df['category_3'].fillna('A',inplace=True)
    hist_df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    hist_df['installments'].replace(-1, np.nan,inplace=True)
    hist_df['installments'].replace(999, np.nan,inplace=True)

    # trim
    hist_df['purchase_amount'] = hist_df['purchase_amount'].apply(lambda x: min(x, 0.8))

    # Y/N to 1/0
    hist_df['authorized_flag'] = hist_df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    hist_df['category_1'] = hist_df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    hist_df['category_3'] = hist_df['category_3'].map({'A':0, 'B':1, 'C':2})
    #交互特征
    hist_df['city_C1'] = ((hist_df['city_id']< 0) + 0).astype(np.str) + '_' + hist_df['category_1'].astype(np.str)
    hist_df['C2_state'] = hist_df['category_2'].astype(np.str) + '_' + hist_df['state_id'].astype(np.str) 
    hist_df['C2_state_subsector'] = hist_df['category_2'].astype(np.str) + '_' + hist_df['state_id'].astype(np.str) + '_' + hist_df['subsector_id'].astype(np.str)
    hist_df['subsector_city'] = hist_df['subsector_id'].astype(np.str) + '_' + ((hist_df['city_id'] < 0) + 0).astype(np.str)
    hist_df['auth_C3'] = hist_df['authorized_flag'].astype(np.str) + '_' + hist_df['category_3'].astype(np.str)

    cateCols = ['city_C1','C2_state','C2_state_subsector','subsector_city','auth_C3']
    hist_df = label_encoding(hist_df,cateCols)

    # datetime features
    hist_df['purchase_date'] = pd.to_datetime(hist_df['purchase_date'])
    hist_df['month'] = hist_df['purchase_date'].dt.month
    hist_df['day'] = hist_df['purchase_date'].dt.day
    hist_df['hour'] = hist_df['purchase_date'].dt.hour
    hist_df['weekofyear'] = hist_df['purchase_date'].dt.weekofyear
    hist_df['weekday'] = hist_df['purchase_date'].dt.weekday
    hist_df['weekend'] = (hist_df['purchase_date'].dt.weekday >=5).astype(int)


    #Christmas : December 25 2017
    hist_df['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Mothers Day: May 14 2017
    hist_df['Mothers_Day_2017']=(pd.to_datetime('2017-06-04')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #fathers day: August 13 2017
    hist_df['fathers_day_2017']=(pd.to_datetime('2017-08-13')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Childrens day: October 12 2017
    hist_df['Children_day_2017']=(pd.to_datetime('2017-10-12')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Valentine's Day : 12th June, 2017
    hist_df['Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Black Friday : 24th November 2017
    hist_df['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

    #2018
    #Mothers Day: May 13 2018
    hist_df['Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

    hist_df['month_diff'] = ((datetime.datetime.today() - hist_df['purchase_date']).dt.days)//30
    hist_df['month_diff'] += hist_df['month_lag']

    # additional features
    df_temp = hist_df.groupby(['card_id','merchant_id'])['purchase_amount'].count().reset_index()
    df_temp.rename(columns={'purchase_amount':'card_merchant_counts_totals'},inplace=True)
    hist_df = hist_df.merge(df_temp,on=['card_id','merchant_id'],how='left')
    
    hist_df['duration'] = hist_df['purchase_amount']*hist_df['month_diff']
    hist_df['duration/Visitcounts'] = hist_df['duration']/hist_df['card_merchant_counts_totals']
    hist_df['duration/sqrtVisits'] = hist_df['duration']/np.sqrt(hist_df['card_merchant_counts_totals'])
    hist_df['durations_log1p_visits'] = hist_df['duration']*np.log1p(hist_df['card_merchant_counts_totals'])
   
    hist_df['purchase/Visitcounts'] = hist_df['purchase_amount']/hist_df['card_merchant_counts_totals']
    hist_df['purchase/sqrtVisits'] = hist_df['purchase_amount']/np.sqrt(hist_df['card_merchant_counts_totals'])
    hist_df['purchase_log1p_visits'] = hist_df['purchase_amount']*np.log1p(hist_df['card_merchant_counts_totals'])
   
    hist_df['purchase_amount_installments'] = hist_df['purchase_amount'] * hist_df['installments']
    hist_df['price'] = hist_df['purchase_amount'] / hist_df['installments']
    hist_df['purchase_amount_authorized_flag'] = hist_df['purchase_amount'] * hist_df['authorized_flag']
    hist_df['purchase_amount_category_1'] = hist_df['purchase_amount'] * hist_df['category_1']
    hist_df['purchase_amount_category_2'] = hist_df['purchase_amount'] * hist_df['category_2']
    
    hist_df['amount_month_ratio'] = hist_df['purchase_amount']/hist_df['month_diff']
    
    hist_df['purchase_lag'] = hist_df['purchase_amount']*hist_df['month_lag']
    hist_df['purchase/month_lag'] = hist_df['purchase_amount']/hist_df['month_lag']
    
    hist_df['installments_month_diff'] = hist_df['installments']*hist_df['month_diff']
    hist_df['installments_month_diff_ratio'] = hist_df['installments']/hist_df['month_diff']
    
    hist_df['purchase_amount_weekend'] = hist_df['purchase_amount']*hist_df['weekend']
    
    df_temp = hist_df.groupby(['card_id'])['purchase_amount'].count().reset_index()
    df_temp.rename(columns={'purchase_amount':'card_visit_counts'},inplace=True)
    hist_df = hist_df.merge(df_temp,on='card_id',how='left')
    
    hist_df['duration/CardVisitcounts'] = hist_df['duration']/hist_df['card_visit_counts']
    hist_df['duration/sqrtCardVisits'] = hist_df['duration']/np.sqrt(hist_df['card_visit_counts'])
    hist_df['durations_log1p_Card_visits'] = hist_df['duration']*np.log1p(hist_df['card_visit_counts'])
   
    hist_df['purchase/CardVisitcounts'] = hist_df['purchase_amount']/hist_df['card_visit_counts']
    hist_df['purchase/sqrtCardVisits'] = hist_df['purchase_amount']/np.sqrt(hist_df['card_visit_counts'])
    hist_df['purchase_log1p_Card_visits'] = hist_df['purchase_amount']*np.log1p(hist_df['card_visit_counts'])

    hist_df['month_diff_installments_card_merchant_counts'] = hist_df['month_diff']*hist_df['installments']*hist_df['card_merchant_counts_totals']
    hist_df['month_diff_installments_card_merchant_counts_ratio'] = hist_df['month_diff']*hist_df['installments']/hist_df['card_merchant_counts_totals']
    
    hist_df['month_diff_installments_card_counts'] = hist_df['month_diff']*hist_df['card_visit_counts']*hist_df['installments']
    hist_df['month_diff_installments_card_counts_ratio'] = hist_df['month_diff']*hist_df['installments']/hist_df['card_visit_counts']

    ##加入month_gap
    hist_df['month_gap'] = ((datetime.datetime.today() - hist_df['purchase_date']).dt.days)//30
    hist_df['purchase_multi_month_gap'] = hist_df['purchase_amount'] * hist_df['month_gap']
    hist_df['purchase_visitcounts_ratio'] = hist_df['purchase_multi_month_gap']/hist_df['card_merchant_counts_totals']
    hist_df['purchase_sqrtvisits_ratio'] = hist_df['purchase_multi_month_gap']/hist_df['card_merchant_counts_totals']
    hist_df['purchase_log1pvisits_ratio'] = hist_df['purchase_multi_month_gap']/hist_df['card_merchant_counts_totals']
    
    hist_df['purchase_card_visits_ratio'] = hist_df['purchase_multi_month_gap']/hist_df['card_visit_counts']
    hist_df['purchase_sqrt_card_visits_ratio']  = hist_df['purchase_multi_month_gap']/hist_df['card_visit_counts']
    hist_df['purchase_log1p_card_visits_ratio'] = hist_df['purchase_multi_month_gap']/hist_df['card_visit_counts']
    
    # reduce memory usage
    hist_df = reduce_mem_usage(hist_df)

    col_unique =['subsector_id', 'merchant_id', 'merchant_category_id']
    col_seas = ['month', 'hour', 'weekofyear', 'weekday', 'day']

    aggs = {}
    for col in ['Christmas_Day_2017','Mothers_Day_2017','fathers_day_2017','Children_day_2017','Valentine_Day_2017','Mothers_Day_2018']:
        hist_df['purchase_amount_%s'%col] = hist_df['purchase_amount'] * hist_df[col]
        aggs[col] = ['min','max','sum','mean','median']
        
    for col in col_unique:
        aggs[col] = ['nunique']

    for col in col_seas:
        aggs[col] = ['nunique', 'mean', 'min', 'max']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','skew']
    aggs['installments'] = ['sum','max','mean','var','skew']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','skew']
    aggs['month_diff'] = ['max','min','mean','var','skew']
    aggs['authorized_flag'] = ['mean']
    aggs['weekend'] = ['mean'] # overwrite
    aggs['weekday'] = ['mean'] # overwrite
    aggs['day'] = ['nunique', 'mean', 'min'] # overwrite
    aggs['category_1'] = ['mean']
    aggs['category_2'] = ['mean']
    aggs['category_3'] = ['mean']
    aggs['card_id'] = ['size','count']
    aggs['price'] = ['sum','mean','max','min','var']
    aggs['Christmas_Day_2017'] = ['mean']
    aggs['Mothers_Day_2017'] = ['mean']
    aggs['fathers_day_2017'] = ['mean']
    aggs['Children_day_2017'] = ['mean']
    aggs['Valentine_Day_2017'] = ['mean']
    aggs['Black_Friday_2017'] = ['mean']
    aggs['Mothers_Day_2018'] = ['mean']
    
    aggs['duration']=['mean','min','max']
    aggs['amount_month_ratio']=['mean','min','max']
    aggs['installments_month_diff'] = ['mean','min','max']
    aggs['installments_month_diff_ratio']=['mean','min','max']
    aggs['purchase_lag'] = ['mean','min','max','median']
    aggs['purchase/month_lag'] = ['mean','min','max','median']
    aggs['purchase_amount_weekend'] = ['mean','min','max','median']
    aggs['duration/Visitcounts'] = ['mean','sum','min','max','median']
    aggs['durations_log1p_visits'] = ['mean','sum','min','max','median']
    aggs['duration/sqrtVisits'] = ['mean','sum','min','max','median']
    
    aggs['purchase/Visitcounts'] = ['mean','sum','min','max','median']
    aggs['purchase_log1p_visits'] = ['mean','sum','min','max','median']
    aggs['purchase/sqrtVisits'] = ['mean','sum','min','max','median']
    
    aggs['purchase_amount_installments'] = ['mean','sum','min','max','median']
    aggs['purchase_amount_authorized_flag'] = ['mean','sum','min','max','median']
    aggs['purchase_amount_category_1'] = ['mean','sum','min','max','median']
    aggs['purchase_amount_category_2'] = ['mean','sum','min','max','median']
    
    aggs['duration/CardVisitcounts'] = ['mean','max','min','median']
    aggs['duration/sqrtCardVisits'] = ['mean','max','min','median']
    aggs['durations_log1p_Card_visits'] = ['mean','max','min','median']
    
    aggs['purchase/CardVisitcounts'] = ['mean','max','min','median']
    aggs['purchase/sqrtCardVisits'] = ['mean','max','min','median']
    aggs['purchase_log1p_Card_visits'] = ['mean','max','min','median'] 
    
    aggs['month_diff_installments_card_merchant_counts'] = ['max','min','mean','median']
    aggs['month_diff_installments_card_merchant_counts_ratio'] = ['max','min','mean','median']
    aggs['month_diff_installments_card_counts'] = ['max','min','mean','median']
    aggs['month_diff_installments_card_counts_ratio'] = ['max','min','mean','median']
    
    aggs['month_gap'] = ['mean','min','max']
    aggs['purchase_multi_month_gap'] = ['mean','max','min','median']
    aggs['purchase_visitcounts_ratio'] = ['mean','max','min','median']
    aggs['purchase_sqrtvisits_ratio'] = ['mean','max','min','median']
    aggs['purchase_log1pvisits_ratio'] = ['mean','max','min','median']
    
    aggs['purchase_card_visits_ratio'] = ['mean','max','min','median']
    aggs['purchase_sqrt_card_visits_ratio'] = ['mean','max','min','median']
    aggs['purchase_log1p_card_visits_ratio'] = ['mean','max','min','median']
    
    cateCols = ['city_C1','C2_state','C2_state_subsector','subsector_city','auth_C3']
    hist_df = label_encoding(hist_df,cateCols)
    for col in cateCols:
        aggs[col] = ['mean']
    
    for col in cateCols+['category_2','category_3']:
        hist_df[col+'_mean'] = hist_df.groupby([col])['purchase_amount'].transform('mean')
        hist_df[col+'_min'] = hist_df.groupby([col])['purchase_amount'].transform('min')
        hist_df[col+'_max'] = hist_df.groupby([col])['purchase_amount'].transform('max')
        hist_df[col+'_sum'] = hist_df.groupby([col])['purchase_amount'].transform('sum')
        aggs[col+'_mean'] = ['mean']

    hist_df = hist_df.reset_index().groupby('card_id').agg(aggs)

    # change column name
    hist_df.columns = pd.Index([e[0] + "_" + e[1] for e in hist_df.columns.tolist()])
    hist_df.columns = ['hist_'+ c for c in hist_df.columns]

    hist_df['hist_purchase_date_diff'] = (hist_df['hist_purchase_date_max']-hist_df['hist_purchase_date_min']).dt.days
    hist_df['hist_purchase_date_average'] = hist_df['hist_purchase_date_diff']/hist_df['hist_card_id_size']
    hist_df['hist_purchase_date_uptonow'] = (datetime.datetime.today()-hist_df['hist_purchase_date_max']).dt.days
    hist_df['hist_purchase_date_uptomin'] = (datetime.datetime.today()-hist_df['hist_purchase_date_min']).dt.days

    # reduce memory usage
    hist_df = reduce_mem_usage(hist_df)

    return hist_df
    
# preprocessing new_merchant_transactions
def new_merchant_transactions(num_rows=None):
    print('....new_merchant.......')
    # load csv
    new_merchant_df = pd.read_csv('./datasets/new_merchant_transactions.csv', nrows=num_rows)

    # fillna
    new_merchant_df['category_2'].fillna(1.0,inplace=True)
    new_merchant_df['category_3'].fillna('A',inplace=True)
    new_merchant_df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    new_merchant_df['installments'].replace(-1, np.nan,inplace=True)
    new_merchant_df['installments'].replace(999, np.nan,inplace=True)

    # trim
    new_merchant_df['purchase_amount'] = new_merchant_df['purchase_amount'].apply(lambda x: min(x, 0.8))

    # Y/N to 1/0
    new_merchant_df['authorized_flag'] = new_merchant_df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    new_merchant_df['category_1'] = new_merchant_df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    new_merchant_df['category_3'] = new_merchant_df['category_3'].map({'A':0, 'B':1, 'C':2}).astype(int)

    #交互特征
    new_merchant_df['city_C1'] = ((new_merchant_df['city_id']< 0) + 0).astype(np.str) + '_' + new_merchant_df['category_1'].astype(np.str)
    new_merchant_df['C2_state'] = new_merchant_df['category_2'].astype(np.str) + '_' + new_merchant_df['state_id'].astype(np.str) 
    new_merchant_df['C2_state_subsector'] = new_merchant_df['category_2'].astype(np.str) + '_' + new_merchant_df['state_id'].astype(np.str) + '_' + new_merchant_df['subsector_id'].astype(np.str)
    new_merchant_df['subsector_city'] = new_merchant_df['subsector_id'].astype(np.str) + '_' + ((new_merchant_df['city_id'] < 0) + 0).astype(np.str)
    new_merchant_df['auth_C3'] = new_merchant_df['authorized_flag'].astype(np.str) + '_' + new_merchant_df['category_3'].astype(np.str)

    cateCols = ['city_C1','C2_state','C2_state_subsector','subsector_city','auth_C3']
    new_merchant_df = label_encoding(new_merchant_df,cateCols)
    
    # datetime features
    new_merchant_df['purchase_date'] = pd.to_datetime(new_merchant_df['purchase_date'])
    new_merchant_df['month'] = new_merchant_df['purchase_date'].dt.month
    new_merchant_df['day'] = new_merchant_df['purchase_date'].dt.day
    new_merchant_df['hour'] = new_merchant_df['purchase_date'].dt.hour
    new_merchant_df['weekofyear'] = new_merchant_df['purchase_date'].dt.weekofyear
    new_merchant_df['weekday'] = new_merchant_df['purchase_date'].dt.weekday
    new_merchant_df['weekend'] = (new_merchant_df['purchase_date'].dt.weekday >=5).astype(int)

    #fathers day: August 13 2017
    new_merchant_df['fathers_day_2017']=(pd.to_datetime('2017-08-13')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Valentine's Day : 12th June, 2017
    new_merchant_df['Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Christmas : December 25 2017
    new_merchant_df['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Childrens day: October 12 2017
    new_merchant_df['Children_day_2017']=(pd.to_datetime('2017-10-12')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Black Friday : 24th November 2017
    new_merchant_df['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    new_merchant_df['Mothers_Day_2017']=(pd.to_datetime('2017-06-04')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

    #Mothers Day: May 13 2018
    new_merchant_df['Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

    new_merchant_df['month_diff'] = ((datetime.datetime.today() - new_merchant_df['purchase_date']).dt.days)//30
    new_merchant_df['month_diff'] += new_merchant_df['month_lag']

    # additional features
    df_temp = new_merchant_df.groupby(['card_id','merchant_id'])['purchase_amount'].count().reset_index()
    df_temp.rename(columns={'purchase_amount':'card_merchant_counts_totals'},inplace=True)
    new_merchant_df = new_merchant_df.merge(df_temp,on=['card_id','merchant_id'],how='left')
    
    new_merchant_df['duration'] = new_merchant_df['purchase_amount']*new_merchant_df['month_diff']
    new_merchant_df['amount_month_ratio'] = new_merchant_df['purchase_amount']/new_merchant_df['month_diff']
    new_merchant_df['installments_month_diff'] = new_merchant_df['installments']*new_merchant_df['month_diff']
    new_merchant_df['installments_month_diff_ratio'] = new_merchant_df['installments']/new_merchant_df['month_diff']
    
    new_merchant_df['duration/Visitcounts'] = new_merchant_df['duration']/new_merchant_df['card_merchant_counts_totals']
    new_merchant_df['duration/sqrtVisits'] = new_merchant_df['duration']/np.sqrt(new_merchant_df['card_merchant_counts_totals'])
    new_merchant_df['durations_log1p_visits'] = new_merchant_df['duration']*np.log1p(new_merchant_df['card_merchant_counts_totals'])
   
    new_merchant_df['purchase/Visitcounts'] = new_merchant_df['purchase_amount']/new_merchant_df['card_merchant_counts_totals']
    new_merchant_df['purchase/sqrtVisits'] = new_merchant_df['purchase_amount']/np.sqrt(new_merchant_df['card_merchant_counts_totals'])
    new_merchant_df['purchase_log1p_visits'] = new_merchant_df['purchase_amount']*np.log1p(new_merchant_df['card_merchant_counts_totals'])
   
    new_merchant_df['purchase_amount_installments'] = new_merchant_df['purchase_amount'] * new_merchant_df['installments']
    new_merchant_df['price'] = new_merchant_df['purchase_amount'] / new_merchant_df['installments']
    new_merchant_df['purchase_amount_authorized_flag'] = new_merchant_df['purchase_amount'] * new_merchant_df['authorized_flag']
    new_merchant_df['purchase_amount_category_1'] = new_merchant_df['purchase_amount'] * new_merchant_df['category_1']
    new_merchant_df['purchase_amount_category_2'] = new_merchant_df['purchase_amount'] * new_merchant_df['category_2']
    new_merchant_df['price'] = new_merchant_df['purchase_amount'] / new_merchant_df['installments']

    new_merchant_df['amount_month_ratio'] = new_merchant_df['purchase_amount']/new_merchant_df['month_diff']
    
    new_merchant_df['purchase_lag'] = new_merchant_df['purchase_amount']*new_merchant_df['month_lag']
    new_merchant_df['purchase/month_lag'] = new_merchant_df['purchase_amount']/new_merchant_df['month_lag']
    
    new_merchant_df['installments_month_diff'] = new_merchant_df['installments']*new_merchant_df['month_diff']
    new_merchant_df['installments_month_diff_ratio'] = new_merchant_df['installments']/new_merchant_df['month_diff']
    
    new_merchant_df['purchase_amount_weekend'] = new_merchant_df['purchase_amount']*new_merchant_df['weekend']
    
    df_temp = new_merchant_df.groupby(['card_id'])['purchase_amount'].count().reset_index()
    df_temp.rename(columns={'purchase_amount':'card_visit_counts'},inplace=True)
    new_merchant_df = new_merchant_df.merge(df_temp,on='card_id',how='left')
    
    new_merchant_df['duration/CardVisitcounts'] = new_merchant_df['duration']/new_merchant_df['card_visit_counts']
    new_merchant_df['duration/sqrtCardVisits'] = new_merchant_df['duration']/np.sqrt(new_merchant_df['card_visit_counts'])
    new_merchant_df['durations_log1p_Card_visits'] = new_merchant_df['duration']*np.log1p(new_merchant_df['card_visit_counts'])
   
    new_merchant_df['purchase/CardVisitcounts'] = new_merchant_df['purchase_amount']/new_merchant_df['card_visit_counts']
    new_merchant_df['purchase/sqrtCardVisits'] = new_merchant_df['purchase_amount']/np.sqrt(new_merchant_df['card_visit_counts'])
    new_merchant_df['purchase_log1p_Card_visits'] = new_merchant_df['purchase_amount']*np.log1p(new_merchant_df['card_visit_counts'])

    new_merchant_df['month_diff_installments_card_merchant_counts'] =  new_merchant_df['month_diff']* new_merchant_df['installments']* new_merchant_df['card_merchant_counts_totals']
    new_merchant_df['month_diff_installments_card_merchant_counts_ratio'] =  new_merchant_df['month_diff']*new_merchant_df['installments']/ new_merchant_df['card_merchant_counts_totals']
    
    new_merchant_df['month_diff_installments_card_counts'] =  new_merchant_df['month_diff']* new_merchant_df['card_visit_counts']* new_merchant_df['installments']
    new_merchant_df['month_diff_installments_card_counts_ratio'] =  new_merchant_df['month_diff']*new_merchant_df['installments']/new_merchant_df['card_visit_counts']

    
    ##加入month_gap
    new_merchant_df['month_gap'] = ((datetime.datetime.today() - new_merchant_df['purchase_date']).dt.days)//30
    new_merchant_df['purchase_multi_month_gap'] = new_merchant_df['purchase_amount'] * new_merchant_df['month_gap']
    new_merchant_df['purchase_visitcounts_ratio'] = new_merchant_df['purchase_multi_month_gap']/new_merchant_df['card_merchant_counts_totals']
    new_merchant_df['purchase_sqrtvisits_ratio'] = new_merchant_df['purchase_multi_month_gap']/new_merchant_df['card_merchant_counts_totals']
    new_merchant_df['purchase_log1pvisits_ratio'] = new_merchant_df['purchase_multi_month_gap']/new_merchant_df['card_merchant_counts_totals']
    
    new_merchant_df['purchase_card_visits_ratio'] = new_merchant_df['purchase_multi_month_gap']/new_merchant_df['card_visit_counts']
    new_merchant_df['purchase_sqrt_card_visits_ratio']  = new_merchant_df['purchase_multi_month_gap']/new_merchant_df['card_visit_counts']
    new_merchant_df['purchase_log1p_card_visits_ratio'] = new_merchant_df['purchase_multi_month_gap']/new_merchant_df['card_visit_counts']
    
    # reduce memory usage
    new_merchant_df = reduce_mem_usage(new_merchant_df)

    col_unique =['subsector_id', 'merchant_id', 'merchant_category_id']
    col_seas = ['month', 'hour', 'weekofyear', 'weekday', 'day']

    aggs = {}
    for col in ['Christmas_Day_2017','Mothers_Day_2017','fathers_day_2017','Children_day_2017','Valentine_Day_2017','Mothers_Day_2018']:
        new_merchant_df['purchase_amount_%s'%col] = new_merchant_df['purchase_amount'] * new_merchant_df[col]
        aggs[col] = ['min','max','sum','mean','median']
        
    for col in col_unique:
        aggs[col] = ['nunique']

    for col in col_seas:
        aggs[col] = ['nunique', 'mean', 'min', 'max']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','skew']
    aggs['installments'] = ['sum','max','mean','var','skew']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','skew']
    aggs['month_diff'] = ['mean','var','skew']
    aggs['weekend'] = ['mean']
    aggs['month'] = ['mean', 'min', 'max']
    aggs['weekday'] = ['mean', 'min', 'max']
    aggs['category_1'] = ['mean']
    aggs['category_2'] = ['mean']
    aggs['category_3'] = ['mean']
    aggs['card_id'] = ['size','count']
    aggs['price'] = ['mean','max','min','var']
    aggs['Christmas_Day_2017'] = ['mean']
    aggs['Children_day_2017'] = ['mean']
    aggs['Black_Friday_2017'] = ['mean']
    aggs['Mothers_Day_2018'] = ['mean']
    
    aggs['duration']=['mean','min','max']
    aggs['amount_month_ratio']=['mean','min','max']
    aggs['installments_month_diff'] = ['mean','min','max']
    aggs['installments_month_diff_ratio']=['mean','min','max']
#     aggs['purchase_exp'] = ['mean','min','max']
    aggs['purchase_lag'] = ['mean','min','max','median']
    aggs['purchase/month_lag'] = ['mean','min','max','median']
    aggs['purchase_amount_weekend'] = ['mean','min','max','median']
    
    aggs['duration/Visitcounts'] = ['mean','sum','min','max','median']
    aggs['durations_log1p_visits'] = ['mean','sum','min','max','median']
    aggs['duration/sqrtVisits'] = ['mean','sum','min','max','median']
    aggs['purchase/Visitcounts'] = ['mean','sum','min','max','median']
    aggs['purchase_log1p_visits'] = ['mean','sum','min','max','median']
    aggs['purchase/sqrtVisits'] = ['mean','sum','min','max','median']
    
    aggs['purchase_amount_installments'] = ['mean','sum','min','max','median']
    aggs['purchase_amount_authorized_flag'] = ['mean','sum','min','max','median']
    aggs['purchase_amount_category_1'] = ['mean','sum','min','max','median']
    aggs['purchase_amount_category_2'] = ['mean','sum','min','max','median']
    
    aggs['duration/CardVisitcounts'] = ['mean','max','min','median']
    aggs['duration/sqrtCardVisits'] = ['mean','max','min','median']
    aggs['durations_log1p_Card_visits'] = ['mean','max','min','median']
    
    aggs['purchase/CardVisitcounts'] = ['mean','max','min','median']
    aggs['purchase/sqrtCardVisits'] = ['mean','max','min','median']
    aggs['purchase_log1p_Card_visits'] = ['mean','max','min','median'] 
    
    aggs['month_diff_installments_card_merchant_counts'] = ['max','min','mean','median']
    aggs['month_diff_installments_card_merchant_counts_ratio'] = ['max','min','mean','median']
    aggs['month_diff_installments_card_counts'] = ['max','min','mean','median']
    aggs['month_diff_installments_card_counts_ratio'] = ['max','min','mean','median']
    
    aggs['month_gap'] = ['mean','min','max']
    aggs['purchase_multi_month_gap'] = ['mean','max','min','median']
    aggs['purchase_visitcounts_ratio'] = ['mean','max','min','median']
    aggs['purchase_sqrtvisits_ratio'] = ['mean','max','min','median']
    aggs['purchase_log1pvisits_ratio'] = ['mean','max','min','median']
    
    aggs['purchase_card_visits_ratio'] = ['mean','max','min','median']
    aggs['purchase_sqrt_card_visits_ratio'] = ['mean','max','min','median']
    aggs['purchase_log1p_card_visits_ratio'] = ['mean','max','min','median']
    
    for col in cateCols:
        aggs[col] = ['mean']
        
    for col in cateCols+['category_2','category_3']:
        new_merchant_df[col+'_mean'] = new_merchant_df.groupby([col])['purchase_amount'].transform('mean')
        new_merchant_df[col+'_min'] = new_merchant_df.groupby([col])['purchase_amount'].transform('min')
        new_merchant_df[col+'_max'] = new_merchant_df.groupby([col])['purchase_amount'].transform('max')
        new_merchant_df[col+'_sum'] = new_merchant_df.groupby([col])['purchase_amount'].transform('sum')
        aggs[col+'_mean'] = ['mean']

    new_merchant_df = new_merchant_df.reset_index().groupby('card_id').agg(aggs)

    # change column name
    new_merchant_df.columns = pd.Index([e[0] + "_" + e[1] for e in new_merchant_df.columns.tolist()])
    new_merchant_df.columns = ['new_'+ c for c in new_merchant_df.columns]

    new_merchant_df['new_purchase_date_diff'] = (new_merchant_df['new_purchase_date_max']-new_merchant_df['new_purchase_date_min']).dt.days
    new_merchant_df['new_purchase_date_average'] = new_merchant_df['new_purchase_date_diff']/new_merchant_df['new_card_id_size']
    new_merchant_df['new_purchase_date_uptonow'] = (datetime.datetime.today()-new_merchant_df['new_purchase_date_max']).dt.days
    new_merchant_df['new_purchase_date_uptomin'] = (datetime.datetime.today()-new_merchant_df['new_purchase_date_min']).dt.days

    # reduce memory usage
    new_merchant_df = reduce_mem_usage(new_merchant_df)

    return new_merchant_df

# additional features
def additional_features(df):
    print('......additional........')
    date_features=['hist_purchase_date_max','hist_purchase_date_min',
                   'new_purchase_date_max', 'new_purchase_date_min']

    for f in date_features:
        df[f] = df[f].astype(np.int64) * 1e-9

    df['card_id_total'] = df['new_card_id_size']+df['hist_card_id_size']
    df['card_id_cnt_total'] = df['new_card_id_count']+df['hist_card_id_count']
    df['card_id_cnt_ratio'] = df['new_card_id_count']/df['hist_card_id_count']
    df['purchase_amount_total'] = df['new_purchase_amount_sum']+df['hist_purchase_amount_sum']
    df['purchase_amount_mean'] = df['new_purchase_amount_mean']+df['hist_purchase_amount_mean']
    df['purchase_amount_max'] = df['new_purchase_amount_max']+df['hist_purchase_amount_max']
    df['purchase_amount_min'] = df['new_purchase_amount_min']+df['hist_purchase_amount_min']
    df['purchase_amount_ratio'] = df['new_purchase_amount_sum']/df['hist_purchase_amount_sum']
    df['month_diff_mean'] = df['new_month_diff_mean']+df['hist_month_diff_mean']
    df['month_diff_ratio'] = df['new_month_diff_mean']/df['hist_month_diff_mean']
    df['month_lag_mean'] = df['new_month_lag_mean']+df['hist_month_lag_mean']
    df['month_lag_max'] = df['new_month_lag_max']+df['hist_month_lag_max']
    df['month_lag_min'] = df['new_month_lag_min']+df['hist_month_lag_min']
    df['category_1_mean'] = df['new_category_1_mean']+df['hist_category_1_mean']
    df['installments_total'] = df['new_installments_sum']+df['hist_installments_sum']
    df['installments_mean'] = df['new_installments_mean']+df['hist_installments_mean']
    df['installments_max'] = df['new_installments_max']+df['hist_installments_max']
    df['installments_ratio'] = df['new_installments_sum']/df['hist_installments_sum']
    df['price_total'] = df['purchase_amount_total'] / df['installments_total']
    df['price_mean'] = df['purchase_amount_mean'] / df['installments_mean']
    df['price_max'] = df['purchase_amount_max'] / df['installments_max']
    df['duration_mean'] = df['new_duration_mean']+df['hist_duration_mean']
    df['duration_min'] = df['new_duration_min']+df['hist_duration_min']
    df['duration_max'] = df['new_duration_max']+df['hist_duration_max']
    df['amount_month_ratio_mean']=df['new_amount_month_ratio_mean']+df['hist_amount_month_ratio_mean']
    df['amount_month_ratio_min']=df['new_amount_month_ratio_min']+df['hist_amount_month_ratio_min']
    df['amount_month_ratio_max']=df['new_amount_month_ratio_max']+df['hist_amount_month_ratio_max']
    df['new_CLV'] = df['new_card_id_count'] * df['new_purchase_amount_sum'] / df['new_month_diff_mean']
    df['hist_CLV'] = df['hist_card_id_count'] * df['hist_purchase_amount_sum'] / df['hist_month_diff_mean']
    df['CLV_ratio'] = df['new_CLV'] / df['hist_CLV']

    return df
df_temp = historical_transactions(num_rows=None)
df_temp = pd.merge(df_temp,new_merchant_transactions(num_rows=None),on='card_id',how='left')
df_temp = additional_features(df_temp)
print('...done.....')

......historical......



Memory usage after optimization is: 4358.90 MB
Decreased by 72.7%



Memory usage after optimization is: 157.40 MB
Decreased by 34.2%
....new_merchant.......



Memory usage after optimization is: 275.20 MB
Decreased by 74.5%
Memory usage after optimization is: 130.82 MB
Decreased by 38.6%
......additional........
...done.....


In [11]:
df_temp.replace([np.inf,-np.inf],-9999,inplace=True)
df_temp.fillna(0,inplace=True)
df_temp.reset_index(inplace=True)

In [12]:
# cols = [_f for _f in df_temp.columns if 'month' in _f]
cols = ['new_purchase_amount_sum','new_purchase_amount_max','new_purchase_amount_min','new_purchase_amount_mean']
df_temp.drop(columns=cols,inplace=True)
df_temp.head()

,card_id,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Mothers_Day_2018_mean,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,...,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio
0,C_ID_00007093c1,12.882812,10.445312,16.765625,15.007812,11.734375,7.468750,13,29,18,...,0.020432,-14.351562,-17.43750,1.718750,-0.098328,-0.116699,0.011932,-0.221354,-954.671814,0.000232
1,C_ID_0001238066,16.656250,0.000000,0.000000,0.227661,0.000000,17.375000,17,65,29,...,0.034485,-13.984375,-18.34375,8.281250,-0.096313,-0.122986,0.057495,-31.970246,-741.518860,0.043115
2,C_ID_0001506ef0,12.453125,9.273438,5.636719,5.894531,9.046875,8.164062,12,28,19,...,0.084473,-15.210938,-18.34375,1.015625,-0.104492,-0.122681,0.007050,-0.241211,-192.726807,0.001252
3,C_ID_0001793786,8.617188,12.812500,14.437500,24.421875,14.921875,0.000000,24,119,48,...,1.599609,-7.246094,-23.75000,26.406250,-0.027985,-0.092712,0.099976,-10.088623,-815.907043,0.012365
4,C_ID_000183fdda,24.875000,0.000000,0.069458,6.167969,0.000000,11.171875,21,73,36,...,0.049469,-13.437500,-19.12500,8.312500,-0.092224,-0.126465,0.057678,-5.993863,-885.520447,0.006769


In [13]:
df_temp = reduce_mem_usage(df_temp)

Memory usage after optimization is: 300.21 MB
Decreased by 33.4%


In [14]:
df_temp.to_csv('./datasets/df_additional_features.csv',index=False)

In [ ]:
df_temp = pd.read_csv('./datasets/df_additional_features.csv')

feats = list(set(df_temp.columns) - set(['target', 'card_id', 'is_test']))
cr = df_temp[feats].corr()

interactions = []
for col in cr.columns:
    inter_col = cr[cr[col] == cr[col].min()].index[0]
    interactions.append([col, inter_col])

index = 1
for inter in interactions:
    df_temp['inter_sum_' + str(index)] = df_temp[inter[0]] + df_temp[inter[1]] 
    df_temp['inter_sub_' + str(index)] = df_temp[inter[0]] - df_temp[inter[1]] 
    df_temp['inter_mult_' + str(index)] = df_temp[inter[0]] * df_temp[inter[1]] 
    df_temp['inter_div_' + str(index)] = df_temp[inter[0]] / df_temp[inter[1]] 
    index += 1

In [21]:
df_temp.head()

,card_id,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Mothers_Day_2018_mean,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,...,inter_mult_306,inter_div_306,inter_sum_307,inter_sub_307,inter_mult_307,inter_div_307,inter_sum_308,inter_sub_308,inter_mult_308,inter_div_308
0,C_ID_00007093c1,12.88,10.445,16.77000,15.0100,11.734,7.470,13,29,18,...,-2.688,-0.6720,1.656,4.344,-4.032000,-2.232143,-2.852,8.848,-17.538300,-0.512479
1,C_ID_0001238066,16.66,0.000,0.00000,0.2277,0.000,17.380,17,65,29,...,-2.874,-0.7185,0.872,3.746,-3.318033,-1.606820,-0.728,2.898,-1.967105,-0.598456
2,C_ID_0001506ef0,12.45,9.270,5.63700,5.8950,9.050,8.164,12,28,19,...,-0.732,-0.7320,1.768,3.232,-1.830000,-3.415301,-1.994,7.670,-13.713216,-0.587334
3,C_ID_0001793786,8.62,12.810,14.44000,24.4200,14.920,0.000,24,119,48,...,-2.952,-0.7380,0.007,2.959,-2.188908,-1.004743,-2.458,4.198,-2.895360,-0.261418
4,C_ID_000183fdda,24.88,0.000,0.06946,6.1680,0.000,11.170,21,73,36,...,-2.772,-0.6930,3.977,6.749,-7.433118,-3.869408,-1.185,3.717,-3.102966,-0.516524
